In [422]:
import bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pprint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep

from urllib.parse import urljoin

from dataclasses import dataclass

import csv

from datetime import datetime, timedelta

In [440]:
url = 'https://www.finam.ru/publications/selection/united/date'

In [164]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver.get("https://www.finam.ru/publications/selection/united/date/2023-03-08/")

In [504]:
def try_load_more(driver, retries=2, retry_sleep=1, timeout=5, debug=False):    
    while retries>0:
        try:
            retries -= 1
            news_id = 'infinity-ui-main'
            news_page, = driver.find_elements(By.ID, news_id)

            show_more_container_id = 'finfin-local-plugin-block-item-publication-list-filter-date-showMore-container'
            show_more_container = WebDriverWait(news_page, timeout).until(EC.presence_of_element_located((By.ID, show_more_container_id)))
#             show_more_container, = driver.find_elements(By.ID, show_more_container_id)

            load_more_class_name = 'pointer'
            load_more = WebDriverWait(show_more_container, timeout).until(EC.element_to_be_clickable((By.CLASS_NAME, load_more_class_name)))
#             load_more, = show_more_container.find_elements(By.CLASS_NAME, load_more_class_name)

            load_more_text = 'Загрузить ещё'
            assert load_more.text == load_more_text, f'{load_more_text} button not found'

            driver.execute_script("arguments[0].click();", load_more)
    #         load_more.click()

            return True
        except Exception as e:
            if debug:
                print(type(e), e)
                print(f"retries left: {retries}")
            sleep(retry_sleep)
    else:
        return False    


In [293]:
try_load_more(driver, debug=True)

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 1
<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0


False

In [505]:
def load_news(driver, load_more_depth=142, debug=False):
    _load_more_depth = load_more_depth
    while try_load_more(driver, debug=debug) and _load_more_depth>0:
        _load_more_depth -= 1
        if debug:
            print(f'load more depth: {load_more_depth-_load_more_depth}')
    if debug:
        print('all news loaded')

In [392]:
load_news(driver, debug=True)

load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver                        0x000000010216cac4 chromedriver + 248516
3   chromedriver                        0x00000001021a2f68 chromedriver + 470888
4   chromedriver                        0x0000000102198260 chromedriver + 426592
5   chromedriver                        0x00000001021d5020 chromedriver + 675872
6   chromedriver                        0x0000000102196b10 chromedriver + 420624
7   chromedriver                        0x0000000102197c30 chromedriver + 425008
8   chromedriver                        0x00000001024f0ae4 chromedriver + 3934948
9   chromedriver     

In [314]:
news_id = 'infinity-ui-main'
news_page, = driver.find_elements(By.ID, news_id)

news_posts_div_id = 'finfin-local-plugin-block-item-publication-list-filter-date-content'
news_posts_div, = news_page.find_elements(By.ID, news_posts_div_id)


for news_post in news_posts_div.find_elements(By.TAG_NAME, 'a'):
    news_post.

SyntaxError: incomplete input (2270439392.py, line 8)

In [506]:
def try_or_none(f):
    def _try_or_none(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return None
    return _try_or_none


@try_or_none
def get_post_href(post):
    attrs = post.attrs
    post_href = attrs['href']
    return post_href
        
@try_or_none
def get_post_text(post):
    return post.p.text


@try_or_none
def get_date(date_name_header_spans):
    return date_name_header_spans[0].text

@try_or_none
def get_author(date_name_header_spans):
    return date_name_header_spans[1].text

def get_post_date_author_title(post):
    post_divs = post.find_all('div')
    date_name_header = post_divs[0]
    date_name_header_spans = date_name_header.find_all('span')
    
    date = get_date(date_name_header_spans)
    
    author_name = get_author(date_name_header_spans)

    post_title = post_divs[1].text
    
    return date, author_name, post_title

def parse_posts(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    posts_div = soup.find("div", {"id": news_posts_div_id})
    posts = posts_div.find_all("a")
    
    for post in posts:
        try:
            post_href = get_post_href(post)
    #         post_link = urljoin(url, post_href)

            date, author_name, post_title = get_post_date_author_title(post)
            
            post_text = get_post_text(post)

            yield {
                'date': date,
                'href': post_href,
                'author_name': author_name,
                'title': post_title,
                'text': post_text,
            }
        except Exception as e:
            print(f'parse of post {post} failed, {type(e)} {e}')
            raise e


In [471]:
for i, post in enumerate(parse_posts(driver)):
    print(i)
    print(post)

0
{'date': '28.04.23', 'href': '/publications/item/nornikel-prinyal-nepopulyarnoe-no-logichnoe-reshenie-20230428-2121/', 'author_name': '\n\t\t\t\t\t\tКоган Евгений\t\t\t\t\t', 'title': '"Норникель" принял непопулярное, но логичное решение ', 'text': 'ГМК на сегодняшний день – одна из наиболее фундаментально сильных и привлекательных промышленных компаний РФ'}
1
{'date': '28.04.23', 'href': '/publications/item/chistaya-pribyl-seligdara-v-2022-godu-upala-bolee-chem-vdvoe-20230428-2108/', 'author_name': '\n\t\t\t\t\t\tFinam.ru\t\t\t\t\t', 'title': 'Чистая прибыль «Селигдара» в 2022 году упала более чем вдвое ', 'text': 'Консолидированная выручка при этом составила 34 722 млн рублей по сравнению с 35 616 млн рублей годом ранее'}
2
{'date': '28.04.23', 'href': '/publications/item/sobytiya-predstoyashchikh-dney-lukoyl-zakroet-reestr-dlya-uchastiya-v-gosa-20230428-2100/', 'author_name': '\n\t\t\t\t\t\tFinam.ru\t\t\t\t\t', 'title': 'События предстоящих дней: «ЛУКОЙЛ» закроет реестр для участи

In [406]:
next(iter(parse_posts(driver)))

{'date': '06.05.23',
 'href': '/publications/item/investory-vse-silnee-veryat-v-defolt-ssha-cherez-3-5-nedel-20230506-2015/',
 'author_name': '\n\t\t\t\t\t\tМитрофанов Никита\t\t\t\t\t',
 'title': 'Инвесторы все сильнее верят в дефолт США через 3-5 недель ',
 'text': 'Джо Байден не хочет повышать потолок госдолга'}

In [507]:
def get_date_ranges(begin, end, load_delta):
    current = begin
    yield current
    while end > current + load_delta:
        current += load_delta
        yield current

In [508]:
finam_date_fmt = '%Y-%m-%d'
def to_finam_datefmt(date):
    return date.strftime(finam_date_fmt)
    
def load_page(driver, load_from, load_to):
    news_url = f'{url}/{to_finam_datefmt(load_from)}/{to_finam_datefmt(load_to)}/'
    print(f'loading {news_url}')
    driver.get(news_url)
    load_news(driver, debug=True)

In [511]:
field_names = ['date','href','author_name','title','text']

def parse_append_csv(driver):
    with open('posts.csv', 'a') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=field_names) 

        csv_writer.writerows(list(parse_posts(driver)))


def process_pages(driver, begin, end, load_delta, reverse=True):
    print(f'processing pages {begin} - {end}')
    date_ranges = list(get_date_ranges(begin, end, load_delta))
    for load_from, load_to in reversed(list(zip(date_ranges[:-1], date_ranges[1:]))):
        load_page(driver, load_from, load_to - timedelta(days=1))
        parse_append_csv(driver)


In [512]:
finam_csv_date_fmt = '%d.%m.%y'

def get_earliest_post_date():
    with open('posts.csv', 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file, fieldnames=field_names) 

        dates = []
        for post_from_csv in csv_reader: 
            post_date = datetime.strptime(post_from_csv['date'], finam_csv_date_fmt)
            dates.append(post_date)

    return min(dates)

In [513]:
try:
    end = get_earliest_post_date()
except:
    end = datetime.now()
begin = end - timedelta(days=365)
load_delta = timedelta(days=7)



process_pages(driver, begin, end, load_delta)

processing pages 2022-05-08 21:54:53.680835 - 2023-05-08 21:54:53.680835
loading https://www.finam.ru/publications/selection/united/date/2023-04-30/2023-05-06/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver                        0x000000010216cac4 chromedriver + 248516
3   chro

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0
all news loaded
loading https://www.finam.ru/publications/selection/united/date/2023-04-02/2023-04-08/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
load more depth: 27
load more depth: 28
load more depth: 29
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   c

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0
all news loaded
loading https://www.finam.ru/publications/selection/united/date/2023-03-05/2023-03-11/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver                        0x000000010216cac4 chromedriver + 248516
3   chromedriver        

load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver                        0x000000010216cac4 chromedriver + 248516
3   chromedriver                        0x00000001021a2f68 chromedriver + 470888
4   chromedriver                        0x0000000102198260 chromedriver + 426592
5   chromedriver                        0x00000001021d5020 chromedriver + 675872
6   chromedriver                        0x0000000102196b10 chromedriver + 420624
7   chromedrive

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0
all news loaded
loading https://www.finam.ru/publications/selection/united/date/2023-01-15/2023-01-21/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver                        0x000000010216cac4 chrome

...

all news loaded
loading https://www.finam.ru/publications/selection/united/date/2022-08-14/2022-08-20/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
load more depth: 27
load more depth: 28
load more depth: 29
load more depth: 30
load more depth: 31
load more depth: 32
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver          

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0
all news loaded
loading https://www.finam.ru/publications/selection/united/date/2022-07-17/2022-07-23/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
load more depth: 27
load more depth: 28
load more depth: 29
load more depth: 30
load more depth: 31
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0
all news loaded
loading https://www.finam.ru/publications/selection/united/date/2022-06-26/2022-07-02/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
load more depth: 27
load more depth: 28
load more depth: 29
load more depth: 30
load more depth: 31
load more depth: 32
load more depth: 33
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chr

<class 'selenium.common.exceptions.TimeoutException'> Message: 

retries left: 0
all news loaded
loading https://www.finam.ru/publications/selection/united/date/2022-05-29/2022-06-04/
load more depth: 1
load more depth: 2
load more depth: 3
load more depth: 4
load more depth: 5
load more depth: 6
load more depth: 7
load more depth: 8
load more depth: 9
load more depth: 10
load more depth: 11
load more depth: 12
load more depth: 13
load more depth: 14
load more depth: 15
load more depth: 16
load more depth: 17
load more depth: 18
load more depth: 19
load more depth: 20
load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
load more depth: 27
load more depth: 28
load more depth: 29
load more depth: 30
load more depth: 31
load more depth: 32
load more depth: 33
load more depth: 34
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriv

load more depth: 21
load more depth: 22
load more depth: 23
load more depth: 24
load more depth: 25
load more depth: 26
load more depth: 27
load more depth: 28
load more depth: 29
load more depth: 30
load more depth: 31
load more depth: 32
load more depth: 33
load more depth: 34
load more depth: 35
load more depth: 36
load more depth: 37
<class 'selenium.common.exceptions.TimeoutException'> Message: 
Stacktrace:
0   chromedriver                        0x000000010251ea88 chromedriver + 4123272
1   chromedriver                        0x00000001024aa778 chromedriver + 3647352
2   chromedriver                        0x000000010216cac4 chromedriver + 248516
3   chromedriver                        0x00000001021a2f68 chromedriver + 470888
4   chromedriver                        0x0000000102198260 chromedriver + 426592
5   chromedriver                        0x00000001021d5020 chromedriver + 675872
6   chromedriver                        0x0000000102196b10 chromedriver + 420624
7   chromedrive

In [502]:
min(dates)

datetime.datetime(2023, 3, 2, 0, 0)

In [132]:
driver.close()